# Utilizing advanced features - Host-specific `"submit_options"`
In this tutorial we will be exploring how to use advanced features of the framework. We will be using common terminology found in the repo's README.md - refer to that for any underlined terms that need clarification. Additionally, we will be building upon the material covered in the [Basic Test Config](./BasicTestConfig.ipynb); please review that tutorial if you haven't already. Anything in `"code-quoted"` format refers specifically to the test config, and anything in <ins>underlined</ins> format refers to specific terminology that can be found in the [README.md](../README.md).


In [ ]:
# Get notebook location
shellReturn = !pwd
notebookDirectory = shellReturn[0]
print( "Working from " + notebookDirectory )

Advanced usage of the json config options `"<match-fqdn>"` under `"submit_options"` will be the focus of this tutorial :


In [ ]:
# Output template file documenting options
from IPython.display import Markdown as md
md( "```jsonc\n" + open( notebookDirectory + "/../.ci/template.json", "r" )
                        .read()
                        .split( "]\n    }," )[1]
                        .split( "\"<may-match-other-host>\"" )[0] + 
   "\n```" )

## Host-Specific `"submit_options"`

One can conditionally control whether `"submit_options"` are applied based on the FQDN ([fully qualified domain name](https://en.wikipedia.org/wiki/Fully_qualified_domain_name)) of the host running the step. The logic to do so is simple but provides incredible flexibility and power in writing <ins>test configs</ins>. 



In [ ]:
# Output code snippet demonstrating feature
from IPython.display import Markdown as md
md( "```python\n  def selectHostSpecificSubmitOptions" + 
     open( notebookDirectory + "/../.ci/SubmitOptions.py", "r" )
                        .read()
                        .split( "def selectHostSpecificSubmitOptions" )[1]
                        .split( "def format" )[0] + 
   "\n```" )

The host argument is always supplied when this function is called in `Step.py` which is originally supplied by the `globalOpts_` (command-line options)

In [ ]:
# Output code snippet demonstrating feature
from IPython.display import Markdown as md
md( "```python\n##### From Step.py #####" + 
     open( notebookDirectory + "/../.ci/Step.py", "r" )
                        .read()
                        .split( "Step.DependencyType( depType )" )[1]
                        .split( "def validate" )[0] + 

   "\n##### From runner.py #####" +
   open( notebookDirectory + "/../.ci/runner.py", "r" )
                        .read()
                        .split( "basename = os.path.splitext( os.path.basename( options.testsConfig ) )[0]" )[1]
                        .split( "success = False" )[0] + 
   "\n```" )


All we do is look at the accummulated `"submit_options"` and override any defaults with the first one that matches our FQDN. You might also notice that we have the ability to force it to appear as if we are a different host. This has some powerful implications that we will cover later, but for now we will us that to show how this all works. As there is no way to predict the hostname of where you might be running this notebook, instead we will say we are running on `tutorials.hpc-workflows.foobar.com`. We'll use the `--forceFQDN/-ff` flag to set our own "hostname" :




In [ ]:
%%bash -s "$notebookDirectory"
$1/../.ci/runner.py $1/../our-config.json -h | \
  tr $'\n' '@' | \
  sed -e 's/[ ]\+-h.*\?directly to stdout/.../g' | \
  sed -e 's/[ ]\+-fs.*/.../g' | \
  tr '@' $'\n'

Let's start with a small set of `"submit_options"` that will be our defaults :

In [ ]:
%%bash -s "$notebookDirectory" 
cat << EOF > $1/../our-config.json
{
  "submit_options" :
  {
    "submission" : "LOCAL",
    "timelimit"  : "00:01:00",
    "arguments"  :
    {
      "data_path" : [ "-p", "/some/local/path/" ]
    }
  },
  "our-test" :
  {
    "steps" : { "our-step0-less-nodes" : { "command" : "./tests/scripts/echo_normal.sh" } }
  }
}
EOF

echo "$( realpath $1/../our-config.json ) :"
cat $1/../our-config.json

$1/../.ci/runner.py $1/../our-config.json -t our-test -fs -i -ff tutorials.hpc-workflows.foobar.com

Looks good, let's start overriding our `"data_path"` <ins>argpack</ins>. Recall that we _could_ override it by providing an appropriate `"submit_options"` at the test or step level, but say we have many steps and tests that will make use of this data path. Likewise, on our `tutorials.hpc-workflows.foobar.com` we have the path set to something specific like `/opt/data/path/`. Instead we want to supply a host-specific designation at the top to be our new default :

In [ ]:
%%bash -s "$notebookDirectory" 
cat << EOF > $1/../our-config.json
{
  "submit_options" :
  {
    "submission" : "LOCAL",
    "timelimit"  : "00:01:00",
    "arguments"  :
    {
      "data_path" : [ "-p", "/some/local/path/" ]
    },
    "tutorials" :
    {
      "arguments" :
      {
        "data_path" : [ "-p", "/opt/data/path" ]
      }
    }
  },
  "our-test" :
  {
    "steps" : { "our-step0-less-nodes" : { "command" : "./tests/scripts/echo_normal.sh" } }
  }
}
EOF

echo "$( realpath $1/../our-config.json ) :"
cat $1/../our-config.json

$1/../.ci/runner.py $1/../our-config.json -t our-test -fs -i -ff tutorials.hpc-workflows.foobar.com

Notice that our input to the command script is now our host-specific `"arguments"`? Also, we need only match the name with python string [operator `in`](https://docs.python.org/3/reference/expressions.html#membership-test-details). If we then want to override this default for this host at the test level we need to match the previous string verbatim :

In [ ]:
%%bash -s "$notebookDirectory" 
cat << EOF > $1/../our-config.json
{
  "submit_options" :
  {
    "submission" : "LOCAL",
    "timelimit"  : "00:01:00",
    "arguments"  :
    {
      "data_path" : [ "-p", "/some/local/path/" ]
    },
    "tutorials" :
    {
      "arguments" :
      {
        "data_path" : [ "-p", "/opt/data/path" ]
      }
    }
  },
  "our-test" :
  {
    "submit_options" :
    {
      "tutorials" :
      {
        "arguments" :
        {
          "data_path" : [ "-p", "/home/user/data/path" ]
        }
      }
    },
    "steps" : { "our-step0-less-nodes" : { "command" : "./tests/scripts/echo_normal.sh" } }
  }
}
EOF

echo "$( realpath $1/../our-config.json ) :"
cat $1/../our-config.json

$1/../.ci/runner.py $1/../our-config.json -t our-test -fs -i -ff tutorials.hpc-workflows.foobar.com

As shown in the .ci/template.json, host-specific `"submit_options"` must first appear in a parent `"submit_options"` entry but do not need the keyword themselves as their unique name counts as that. 

All the rules of `"submit_options"` (inheritance, overriding, etc.) apply here as well to their respective unique name entry. Thus, if we were to have written `"hpc-workflows"` as our host-specific name match for trying to override at the test level it would count as a separate entry. And since the selection process only takes the first one found that matches (entry names are preserved in the order they appear in the config) we would not get the expected results. 

If this doesn't make sense try running the following config and seeing what the data path is set to when `"tutorials"` host-specifics exists as well as when it is removed :
```json
{
  "submit_options" :
  {
    "submission" : "LOCAL",
    "timelimit"  : "00:01:00",
    "arguments"  :
    {
      "data_path" : [ "-p", "/some/local/path/" ]
    },
    "tutorials" :
    {
      "arguments" :
      {
        "data_path" : [ "-p", "/opt/data/path" ]
      }
    }
  },
  "our-test" :
  {
    "submit_options" :
    {
      "hpc-workflows" :
      {
        "arguments" :
        {
          "data_path" : [ "-p", "/home/user/data/path" ]
        }
      }
    },
    "steps" : { "our-step0-less-nodes" : { "command" : "./tests/scripts/echo_normal.sh" } }
  }
}
```

Since we can actually force-set the pressumed FQDN, we can technically use these host-specific `"submit_options"` to act more like selectable configurations, essentially. This is by design and is encouraged if used appropriately. The selection based on FQDN is more of a convenience to be the default selection criteria.